#  Regularized MLR (OLS / Ridge / Lasso / ElasticNet) 

#### Imports — core utils, data wrangling, ML, and stats

In [85]:
# === SPEED HEADER (7950X, sklearn CV parallel) ===============================
import os
# Prevent BLAS oversubscription; we’ll parallelize at sklearn/joblib level
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["NUMEXPR_MAX_THREADS"] = "1"

# Parallelism knob
N_JOBS = 18

In [86]:
# Core + utils
import os
from math import sqrt

# Data handling
import numpy as np
import pandas as pd

# ML (scikit-learn)
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed          

# Stats (statsmodels)
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from threadpoolctl import threadpool_limits  

#### Data paths, load, and prep

In [87]:
# Paths (aligned with earlier prep)
SAVE_DIR   = '../Extended Parametric Regression Files+Plots'
TRAIN_CSV  = f'{SAVE_DIR}/train.csv'
TEST_CSV   = f'{SAVE_DIR}/test.csv'
FOLDS_NPY  = f'{SAVE_DIR}/train_folds.npy'

# Load splits
df_train = pd.read_csv(TRAIN_CSV)
df_test  = pd.read_csv(TEST_CSV)
fold_assignments = np.load(FOLDS_NPY)

# Feature/target setup (physics-consistent)
raw_feats  = ['distance','frequency','c_walls','w_walls',
              'co2','humidity','pm25','pressure','temperature','snr']
target_col = 'PL'

# Train/test matrices
Xtr_raw = df_train[raw_feats].copy()
ytr_pl  = df_train[target_col].astype(float).values
Xte_raw = df_test[raw_feats].copy()
yte_pl  = df_test[target_col].astype(float).values

#### Physics-consistent linearization + helpers + model specs

In [88]:
# Linearize (Friis-adjusted): y_adj = PL - 20*log10(f)
d0 = 1.0
def z_of_d(d): 
    return 10.0*np.log10(np.clip(d.astype(float), 1e-6, None)/d0)
def f_term(f):
    return 20.0*np.log10(np.clip(f.astype(float), 1e-12, None))

# Adjusted targets
ftr_tr, ftr_te = f_term(Xtr_raw['frequency'].values), f_term(Xte_raw['frequency'].values)
ytr_adj, yte_adj = ytr_pl - ftr_tr, yte_pl - ftr_te

# Linear feature maps
cols = ['z_d','c_walls','w_walls','co2','humidity','pm25','pressure','temperature','snr']
Xtr_lin = pd.DataFrame({
    'z_d': z_of_d(Xtr_raw['distance'].values),
    'c_walls': Xtr_raw['c_walls'].values,
    'w_walls': Xtr_raw['w_walls'].values,
    'co2': Xtr_raw['co2'].values,
    'humidity': Xtr_raw['humidity'].values,
    'pm25': Xtr_raw['pm25'].values,
    'pressure': Xtr_raw['pressure'].values,
    'temperature': Xtr_raw['temperature'].values,
    'snr': Xtr_raw['snr'].values
}, columns=cols).values
Xte_lin = pd.DataFrame({
    'z_d': z_of_d(Xte_raw['distance'].values),
    'c_walls': Xte_raw['c_walls'].values,
    'w_walls': Xte_raw['w_walls'].values,
    'co2': Xte_raw['co2'].values,
    'humidity': Xte_raw['humidity'].values,
    'pm25': Xte_raw['pm25'].values,
    'pressure': Xte_raw['pressure'].values,
    'temperature': Xte_raw['temperature'].values,
    'snr': Xte_raw['snr'].values
}, columns=cols).values

# Param labels (for reporting)
param_names = [
    'PL(d0) [dB]', 'Path loss exponent (n)',
    'Brick Wall Loss (L_c) [dB]', 'Wood Wall Loss (L_w) [dB]',
    'CO2 coef. [dB/unit]', 'Humidity coef. [dB/%]',
    'PM2.5 coef. [dB/µg/m³]', 'Pressure coef. [dB/hPa]',
    'Temp. coef. [dB/°C]', 'SNR scaling (k_snr)'
]

# Helpers
def unscale_coefficients(pipeline):
    """Undo StandardScaler effect → coeffs in original units."""
    steps = pipeline.named_steps
    est = steps.get('ridge') or steps.get('lasso') or steps.get('elasticnet') or steps.get('linearregression')
    if 'standardscaler' not in steps:
        return float(est.intercept_), est.coef_.astype(float).copy()
    scaler = steps['standardscaler']
    beta_scaled = est.coef_.astype(float)
    mu, sig = scaler.mean_, scaler.scale_
    beta_orig = beta_scaled / sig
    intercept_orig = float(est.intercept_ - np.sum(beta_scaled * mu / sig))
    return intercept_orig, beta_orig

def fold_indices(folds, k):
    val_idx = np.where(folds == k)[0]
    tr_idx  = np.where(folds != k)[0]
    return tr_idx, val_idx

def rmse_r2_on_PL(y_true_pl, y_pred_adj, fterm):
    """Score in PL-domain (add back freq term)."""
    y_pred_pl = y_pred_adj + fterm
    rmse = sqrt(mean_squared_error(y_true_pl, y_pred_pl))
    r2   = r2_score(y_true_pl, y_pred_pl)
    return rmse, r2

# Model factories
def make_OLS(_): return make_pipeline(LinearRegression())
def make_Ridge(cfg): return make_pipeline(StandardScaler(), Ridge(alpha=cfg["alpha"], random_state=42))
def make_Lasso(cfg): return make_pipeline(StandardScaler(), Lasso(alpha=cfg["alpha"], max_iter=20000, random_state=42))
def make_ElasticNet(cfg): return make_pipeline(
    StandardScaler(),
    ElasticNet(alpha=cfg["alpha"], l1_ratio=cfg["l1_ratio"], max_iter=20000, random_state=42)
)

# Grids
ridge_grid = [dict(alpha=a) for a in np.logspace(-4, 3, 15)]
lasso_grid = [dict(alpha=a) for a in np.logspace(-4, 1, 15)]
enet_grid  = [dict(alpha=a, l1_ratio=r) for a in np.logspace(-4, 1, 10) for r in (0.2, 0.5, 0.8)]

# Spec list
specs = [
    ("OLS",        make_OLS,        [dict()]),
    ("Ridge",      make_Ridge,      ridge_grid),
    ("Lasso",      make_Lasso,      lasso_grid),
    ("ElasticNet", make_ElasticNet, enet_grid),
]

#### CV search → best hyperparams → refit on full train → evaluate on test

In [89]:
# K-fold CV over each spec; pick by mean Val RMSE; refit on all train; score on test
results = []
K = int(np.max(fold_assignments)) + 1

# Precompute indices once (saves a bit)
folds = [fold_indices(fold_assignments, k) for k in range(K)]

from joblib import Parallel, delayed

def eval_cfg(factory, cfg, folds):
    tr_rmse_list, val_rmse_list, tr_r2_list, val_r2_list = [], [], [], []
    for tr_idx, val_idx in folds:
        X_tr, X_val = Xtr_lin[tr_idx], Xtr_lin[val_idx]
        y_tr, y_val = ytr_adj[tr_idx], ytr_adj[val_idx]
        ypl_tr, ypl_val = ytr_pl[tr_idx], ytr_pl[val_idx]
        f_tr,  f_val  = ftr_tr[tr_idx],  ftr_tr[val_idx]

        pipe = factory(cfg)
        pipe.fit(X_tr, y_tr)

        y_tr_pred_adj = pipe.predict(X_tr)
        rmse_tr, r2_tr = rmse_r2_on_PL(ypl_tr, y_tr_pred_adj, f_tr)
        tr_rmse_list.append(rmse_tr); tr_r2_list.append(r2_tr)

        y_val_pred_adj = pipe.predict(X_val)
        rmse_val, r2_val = rmse_r2_on_PL(ypl_val, y_val_pred_adj, f_val)
        val_rmse_list.append(rmse_val); val_r2_list.append(r2_val)

    return {
        "cfg": cfg,
        "rmse_train_mean": float(np.mean(tr_rmse_list)), "rmse_train_sd": float(np.std(tr_rmse_list)),
        "rmse_val_mean":   float(np.mean(val_rmse_list)), "rmse_val_sd":   float(np.std(val_rmse_list)),
        "r2_train_mean":   float(np.mean(tr_r2_list)),    "r2_train_sd":    float(np.std(tr_r2_list)),
        "r2_val_mean":     float(np.mean(val_r2_list)),   "r2_val_sd":      float(np.std(val_r2_list)),
    }

for name, factory, grid in specs:
    if len(grid) == 1:  # OLS: no sweep
        grid_results = [eval_cfg(factory, grid[0], folds)]
    else:
        # Parallelize across configs; BLAS is single-threaded (from header), so this scales well.
        grid_results = Parallel(n_jobs=N_JOBS, backend="threading", prefer="threads", verbose=0)(
            delayed(eval_cfg)(factory, cfg, folds) for cfg in grid
        )

    best_res = min(grid_results, key=lambda r: r["rmse_val_mean"])
    best_cfg, best_cv = best_res["cfg"], {k: v for k, v in best_res.items() if k != "cfg"}

    # Refit on all training data with best hyperparams
    final_pipe = factory(best_cfg)
    final_pipe.fit(Xtr_lin, ytr_adj)

    # Test metrics (PL domain)
    yte_pred_adj = final_pipe.predict(Xte_lin)
    test_rmse, test_r2 = rmse_r2_on_PL(yte_pl, yte_pred_adj, ftr_te)

    # Coefficients back in original units
    intercept_orig, beta_orig = unscale_coefficients(final_pipe)
    coeffs = np.concatenate(([intercept_orig], beta_orig))
    coeffs_series = pd.Series(coeffs, index=param_names, name=name)

    results.append({
        "model": name,
        "best_cfg": best_cfg,
        "cv": best_cv,
        "test": {"rmse": float(test_rmse), "r2": float(test_r2)},
        "coeffs": coeffs_series
    })

####  CV summary (train/val RMSE + R²)

In [90]:
def fmt(mu, sd): 
    return f"{mu:.4f} ± {sd:.4f}"

# Collect per-model CV stats
cv_rows = []
for res in results:
    cv = res['cv']
    cv_rows.append({
        "Model":       res['model'],
        "RMSE (Train)": fmt(cv["rmse_train_mean"], cv["rmse_train_sd"]),
        "RMSE (Val)":   fmt(cv["rmse_val_mean"],   cv["rmse_val_sd"]),
        "R2 (Train)":   fmt(cv["r2_train_mean"],   cv["r2_train_sd"]),
        "R2 (Val)":     fmt(cv["r2_val_mean"],     cv["r2_val_sd"]),
    })

# Wrap in DataFrame for organized display
cv_df = pd.DataFrame(cv_rows)
print("\n=== Cross-Validation Results (training set) ===")
display(cv_df)


=== Cross-Validation Results (training set) ===


,Model,RMSE (Train),RMSE (Val),R2 (Train),R2 (Val)
0,OLS,8.0733 ± 0.0035,8.0734 ± 0.0141,0.8172 ± 0.0002,0.8171 ± 0.0010
1,Ridge,8.0733 ± 0.0035,8.0734 ± 0.0141,0.8172 ± 0.0002,0.8171 ± 0.0010
2,Lasso,8.0733 ± 0.0035,8.0734 ± 0.0141,0.8172 ± 0.0002,0.8171 ± 0.0010
3,ElasticNet,8.0733 ± 0.0035,8.0734 ± 0.0141,0.8172 ± 0.0002,0.8171 ± 0.0010


#### Test summary (final fit on full train)

In [91]:
# Collect per-model test scores
test_rows = []
for res in results:
    te = res['test']
    test_rows.append({
        "Model": res['model'],
        "Test RMSE": f"{te['rmse']:.4f}",
        "Test R2":   f"{te['r2']:.4f}"
    })

# Frame + display
test_df = pd.DataFrame(test_rows)
print("\n=== Test (final fit on all train) ===")
display(test_df)


=== Test (final fit on all train) ===


,Model,Test RMSE,Test R2
0,OLS,8.0642,0.8172
1,Ridge,8.0642,0.8172
2,Lasso,8.0642,0.8172
3,ElasticNet,8.0642,0.8172


#### Coefficient table (all models, final fits, original units)

In [92]:
# Collect coeffs side by side
coef_df = pd.concat([res['coeffs'] for res in results], axis=1)
coef_df.columns = [res['model'] for res in results]

print("\n=== Harmonized Coefficients (final all-train fits, original units) ===")
display(coef_df)


=== Harmonized Coefficients (final all-train fits, original units) ===


,OLS,Ridge,Lasso,ElasticNet
PL(d0) [dB],2.981685,2.981891,2.974742,2.977869
Path loss exponent (n),3.846162,3.846137,3.846252,3.846027
Brick Wall Loss (L_c) [dB],6.872957,6.872970,6.872652,6.872814
Wood Wall Loss (L_w) [dB],2.012571,2.012599,2.012384,2.012666
CO2 coef. [dB/unit],-0.002361,-0.002361,-0.002360,-0.002361
Humidity coef. [dB/%],-0.087425,-0.087425,-0.087401,-0.087399
PM2.5 coef. [dB/µg/m³],-0.100704,-0.100701,-0.100670,-0.100653
Pressure coef. [dB/hPa],-0.009539,-0.009539,-0.009527,-0.009530
Temp. coef. [dB/°C],-0.146796,-0.146797,-0.146761,-0.146774
SNR scaling (k_snr),-2.034727,-2.034723,-2.034704,-2.034677


#### Chosen hyperparameters (picked by mean Val RMSE)

In [93]:
# Collect best config per model
hp_rows = []
for res in results:
    cfg = res['best_cfg']
    hp_rows.append({"Model": res['model'], **({} if cfg is None else cfg)})

# Frame + display
hp_df = pd.DataFrame(hp_rows).fillna("—")
print("\n=== Chosen Hyperparameters (by mean Val RMSE) ===")
display(hp_df)


=== Chosen Hyperparameters (by mean Val RMSE) ===


,Model,alpha,l1_ratio
0,OLS,—,—
1,Ridge,3.162278,—
2,Lasso,0.0001,—
3,ElasticNet,0.0001,0.8


##  ANOVA for MLR (physics form)

#### OLS analysis frames + formulas (PL adj by Friis; linear predictors)

In [94]:
# Build analysis DataFrame (PL_adj + linear drivers)
dfA = pd.DataFrame({
    "PL_adj":       df_train[target_col].astype(float).values - f_term(df_train["frequency"].values),
    "z_d":          z_of_d(df_train["distance"].values),
    "c_walls":      df_train["c_walls"].values,
    "w_walls":      df_train["w_walls"].values,
    "co2":          df_train["co2"].values,
    "humidity":     df_train["humidity"].values,
    "pm25":         df_train["pm25"].values,
    "pressure":     df_train["pressure"].values,
    "temperature":  df_train["temperature"].values,
    "snr":          df_train["snr"].values,
})

# Formulas
full_terms        = "z_d + c_walls + w_walls + co2 + humidity + pm25 + pressure + temperature + snr"
formula_full      = f"PL_adj ~ {full_terms}"
struct_terms      = "z_d + c_walls + w_walls"
formula_struct    = f"PL_adj ~ {struct_terms}"
env_terms         = "co2 + humidity + pm25 + pressure + temperature"
formula_structenv = f"PL_adj ~ {struct_terms} + {env_terms}"

# Fit OLS (HC3 robust SEs)
model_full   = smf.ols(formula=formula_full,      data=dfA).fit(cov_type="HC3")
model_struct = smf.ols(formula=formula_struct,    data=dfA).fit(cov_type="HC3")
model_env    = smf.ols(formula=formula_structenv, data=dfA).fit(cov_type="HC3")


#### ANOVA (Type II, HC3 robust)

In [95]:
# Run Type-II ANOVA (HC3)
anova_type2 = anova_lm(model_full, typ=2, robust="hc3")

print("\n=== Type-II ANOVA (HC3 robust) — additive linear model ===")
display(anova_type2)


=== Type-II ANOVA (HC3 robust) — additive linear model ===


,sum_sq,df,F,PR(>F)
z_d,2.286201e+07,1.0,350755.990378,0.000000e+00
c_walls,1.684031e+07,1.0,258369.119849,0.000000e+00
w_walls,3.015616e+06,1.0,46266.506511,0.000000e+00
co2,9.726505e+04,1.0,1492.270167,0.000000e+00
humidity,3.662827e+05,1.0,5619.620572,0.000000e+00
pm25,5.892599e+04,1.0,904.060490,1.497585e-198
pressure,9.318504e+03,1.0,142.967338,5.998596e-33
temperature,4.066776e+05,1.0,6239.372413,0.000000e+00
snr,1.269239e+07,1.0,194730.480352,0.000000e+00
Residual,8.743282e+07,1341421.0,NaN,NaN


#### ANOVA (Type III, HC3 robust)

In [96]:
# Run Type-III ANOVA (HC3)
anova_type3 = anova_lm(model_full, typ=3, robust="hc3")

print("\n=== Type-III ANOVA (HC3 robust) — additive linear model ===")
display(anova_type3)


=== Type-III ANOVA (HC3 robust) — additive linear model ===


,sum_sq,df,F,PR(>F)
Intercept,7.044036e+03,1.0,108.071757,2.597965e-25
z_d,2.286201e+07,1.0,350755.990378,0.000000e+00
c_walls,1.684031e+07,1.0,258369.119849,0.000000e+00
w_walls,3.015616e+06,1.0,46266.506511,0.000000e+00
co2,9.726505e+04,1.0,1492.270167,0.000000e+00
humidity,3.662827e+05,1.0,5619.620572,0.000000e+00
pm25,5.892599e+04,1.0,904.060490,1.497585e-198
pressure,9.318504e+03,1.0,142.967338,5.998596e-33
temperature,4.066776e+05,1.0,6239.372413,0.000000e+00
snr,1.269239e+07,1.0,194730.480352,0.000000e+00


#### Nested partial-F tests (RSS-based, classical)

In [97]:
# Refit non-robust (needed for compare_f_test → uses RSS/df)
model_full_nr   = smf.ols(formula=formula_full,      data=dfA).fit()
model_struct_nr = smf.ols(formula=formula_struct,    data=dfA).fit()
model_env_nr    = smf.ols(formula=formula_structenv, data=dfA).fit()

# (A) struct → +env
F_A, p_A, _ = model_full_nr.compare_f_test(model_struct_nr)
df_num_A = model_full_nr.df_model - model_struct_nr.df_model
df_den_A = model_full_nr.df_resid
partial_eta2_A = (F_A * df_num_A) / (F_A * df_num_A + df_den_A)

# (B) +env → +snr
F_B, p_B, _ = model_full_nr.compare_f_test(model_env_nr)
df_num_B = model_full_nr.df_model - model_env_nr.df_model
df_den_B = model_full_nr.df_resid
partial_eta2_B = (F_B * df_num_B) / (F_B * df_num_B + df_den_B)

print("\n=== Nested block tests (partial-F on RSS) ===")
print(f"(A) Struct → +Env:  F = {F_A:.4f}, df = ({int(df_num_A)}, {int(df_den_A)}), "
      f"p = {p_A:.3e}, partial η² = {partial_eta2_A:.4f}")
print(f"(B) +Env   → +SNR:  F = {F_B:.4f}, df = ({int(df_num_B)}, {int(df_den_B)}), "
      f"p = {p_B:.3e}, partial η² = {partial_eta2_B:.4f}")


=== Nested block tests (partial-F on RSS) ===
(A) Struct → +Env:  F = 174594.4439, df = (6, 1341421), p = 0.000e+00, partial η² = 0.4385
(B) +Env   → +SNR:  F = 1022513.6116, df = (1, 1341421), p = 0.000e+00, partial η² = 0.4325


#### HC3 coefficient table (interpretable params)

In [98]:
# Extract HC3-robust coefficients + CIs
coef_tbl = model_full.get_robustcov_results(cov_type="HC3").summary2().tables[1].copy()
coef_tbl.rename(columns={
    "Coef.": "coef",
    "Std.Err.": "std_err",
    "P>|t|": "pval",
    "[0.025": "ci_low",
    "0.975]": "ci_high"
}, inplace=True)

print("\n=== Coefficients (HC3 robust) — key parameters ===")
display(coef_tbl.loc[['Intercept','z_d','c_walls','w_walls',
                      'co2','humidity','pm25','pressure','temperature','snr']])


=== Coefficients (HC3 robust) — key parameters ===


,coef,std_err,z,P>|z|,ci_low,ci_high
Intercept,2.981685,0.286818,10.395757,2.592213e-25,2.419533,3.543837
z_d,3.846162,0.006494,592.246562,0.000000e+00,3.833434,3.858890
c_walls,6.872957,0.013521,508.300226,0.000000e+00,6.846456,6.899459
w_walls,2.012571,0.009357,215.096505,0.000000e+00,1.994232,2.030909
co2,-0.002361,0.000061,-38.629913,0.000000e+00,-0.002481,-0.002241
humidity,-0.087425,0.001166,-74.964129,0.000000e+00,-0.089711,-0.085140
pm25,-0.100704,0.003349,-30.067599,1.285646e-198,-0.107269,-0.094140
pressure,-0.009539,0.000798,-11.956895,5.975473e-33,-0.011103,-0.007976
temperature,-0.146796,0.001858,-78.989698,0.000000e+00,-0.150439,-0.143154
snr,-2.034727,0.004611,-441.282767,0.000000e+00,-2.043764,-2.025689
